In [ ]:
!pip install tensorflow

In [ ]:
!pip install PIL

In [ ]:
!pip install scipy

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [5]:
# Parameter
IMG_SIZE = (224,224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001
FINE_TUNE_EPOCHS = 20  # Tambahan epoch untuk fine-tuning
FINE_TUNE_LR = 1e-5    # Learning rate untuk fine-tuning

# Direktori dataset
train_dir = "train"
test_dir = "test"

# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

Found 6881 images belonging to 116 classes.
Found 1292 images belonging to 116 classes.


In [ ]:
# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar untuk transfer learning

# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(256, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

# Kompilasi model untuk transfer learning
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Callback
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Pelatihan awal (transfer learning)
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

c:\Users\MAHASISWA\Documents\GitHub\Nusantara_Culture_Database\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 286s 1s/step - accuracy: 0.1971 - loss: 3.6771 - val_accuracy: 0.5132 - val_loss: 1.6953 - learning_rate: 0.0010
Epoch 2/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 235s 1s/step - accuracy: 0.5058 - loss: 1.7494 - val_accuracy: 0.6308 - val_loss: 1.2766 - learning_rate: 0.0010
Epoch 3/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 215s 994ms/step - accuracy: 0.5944 - loss: 1.3635 - val_accuracy: 0.6695 - val_loss: 1.0850 - learning_rate: 0.0010
Epoch 4/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - accuracy: 0.6407 - loss: 1.1968 - val_accuracy: 0.7190 - val_loss: 0.9327 - learning_rate: 0.0010
Epoch 5/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 216s 999ms/step - accuracy: 0.6641 - loss: 1.1200 - val_accuracy: 0.7423 - val_loss: 0.8890 - learning_rate: 0.0010
Epoch 6/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.6883 - loss: 1.0063 - val_accuracy: 0.7438 - val_loss: 0.8890 - learning_rate: 0.0010
Epoch 7/50
216/216 ━━━━━━━━━━━━━━━━━━━━ 213s 985ms/step - accuracy: 0.6971 -

In [ ]:
model.save("model_50_epochs.h5") # Menyimpan model ke file my_model.h5

In [ ]:
# Fine-tuning
# Buka beberapa lapisan terakhir dari base model
base_model.trainable = True
for layer in base_model.layers[:-20]:  # Bekukan semua kecuali 20 lapisan terakhir
    layer.trainable = False

# Kompilasi ulang model dengan learning rate yang lebih kecil
model.compile(optimizer=Adam(learning_rate=FINE_TUNE_LR),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Fine-tuning model
history_fine_tune = model.fit(
    train_generator,
    epochs=FINE_TUNE_EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

# Evaluasi model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")
model.save("my_model.h5")  # Menyimpan model ke file my_model.h5